# Preliminars

## 1 Conecting reliably

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [ ]:
def getHtml(url):
  try:
    html = urlopen(url)
  except HTTPError as e:
    print(e)
    return None

  try:
    bsObj = BeautifulSoup(html)
    title = bsObj.body.h1
  except AttributeError as e:
    return None
  
  return bsObj

In [ ]:
html = getHtml("https://www.lonabol.gob.bo/web2/")
if html is None:
  print("URL is not found")

array1 = html.find('img').children
#array1 = html.find('head').style.next_siblings
#
#for data in html.find('table'):
#  print(data)

##Retriven data with regular expressions

In [ ]:
from urllib.request import re

images =  html.findAll("img", {'src':re.compile("assets\/img/solidaria/ays.*\.png")}) + html.findAll("img", {'src':re.compile("assets\/img/solidaria/as.*\.png") }) + html.findAll("img", {'src':re.compile("assets\/img/solidaria/aynisoli.*\.png") })
print(images)
for image in images:
  print(image['src'])

[<img alt="" class="img-fluid" src="assets/img/solidaria/ays1.png"/>, <img alt="" class="img-fluid" src="assets/img/solidaria/as4.png"/>, <img alt="" class="img-fluid" src="assets/img/solidaria/asentrerios2.png"/>, <img alt="" class="img-fluid" src="assets/img/solidaria/ascbba.png"/>, <img alt="" class="img-fluid" src="assets/img/solidaria/aynisoli.png"/>]
assets/img/solidaria/ays1.png
assets/img/solidaria/as4.png
assets/img/solidaria/asentrerios2.png
assets/img/solidaria/ascbba.png
assets/img/solidaria/aynisoli.png


#API tracking

In [2]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from urllib.request import urlopen
from urllib.error import HTTPError

##Realiably requests

In [4]:
def getHtml(url):
  try:
    html = urlopen(url)
  except HTTPError as e:
    print(e)
    return None

  try:
    response = requests.get(url)
    html = response.text
    html = BeautifulSoup(html, 'html.parser')
  except AttributeError as e:
    return None
  
  return html

## URL target

In [5]:
url ='https://www.programmableweb.com/category/tools/api'

##Extracting data APIs

In [ ]:
count = 0 
dict_apis = {}
while True:
  soup = getHtml(url)
  if soup is None:
    print("URL is not found")
    break
  rows = soup.find_all('tr', {'class':'odd'}) + soup.find_all('tr', {'class':'even'}) 

  for row in rows:
    API_name , API_URL = row.find('a').text, row.find('a').get('href')
    API_category = row.find('td', {'class':'views-field views-field-field-article-primary-category'}).text
    API_description =  row.find('td', {'class':'views-field views-field-field-api-description'}).text
    dict_apis[count] = [ API_name,API_URL, API_category, API_description ]
    count+=1

  url_next = soup.find('a', {'id':'pager_id_apis_all'})
  if url_next != None:
    if url_next.get('href'):
      url = 'https://www.programmableweb.com' + url_next.get('href')
      print(url)
    else:
      break
  else:
    break


print(count)
df_apis = pd.DataFrame.from_dict(dict_apis, orient='index', columns=['API_name','API_URL','API_category','API_description'] )
df_apis.head()
df_apis.to_csv('api_data')

https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=1
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=2
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=3
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=4
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=5
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=6
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=7
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=8
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=9
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=10
https://www.programmableweb.com/category/tools/api?pw_view_display_id=apis_all&page=11
https://www.programmableweb.com/category/tools/api?p

In [ ]:
df_apis.head()